In [2]:
from geopy.geocoders import Nominatim
import pandas as pd
import requests
import json
import time
import random
from datetime import datetime
import csv
import os
import logging


logging.basicConfig(filename='weather_process.log', encoding='utf-8', level=logging.INFO)

In [2]:
# 若提示不存在geopy库，则执行下面的代码安装
# !pip install geopy

# 数据预处理


目   的：将航班数据与 **飞机型号** 和 **出发城市天气** 数据相关联，形成大表支持建模


数据范围：2021年7月 至 2022年7月

由于原始数据过大，故使用pd.sample方法,抽取每个月份10%的数据，组合成整体数据进行处理

## 各个月份数据整合

In [3]:
raw_data_folder = 'raw_data'
sample_rate = 0.1
#file_name_base  = 'On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_'

In [4]:
raw_data_sets = os.listdir(raw_data_folder)
raw_data_sets = [i for i in raw_data_sets if '.ipynb' not in i]


combin_df = pd.DataFrame()


for file in raw_data_sets:
    print("Processing [{}]...".format(file))
    unit = pd.read_csv(os.path.join(raw_data_folder,file), low_memory=False, dtype='unicode')
    original_len = len(unit)
    
    #随机取10%
    unit = unit.sample(frac=sample_rate, random_state=1)
    sample_len = len(unit)
    
    print('Sampled {} data'.format(sample_len))
    combin_df = combin_df.append(unit)

Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2021_10.csv]...
Sampled 59537 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2021_11.csv]...
Sampled 57669 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2021_12.csv]...
Sampled 58024 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2021_7.csv]...
Sampled 61570 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2021_8.csv]...
Sampled 61149 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2021_9.csv]...
Sampled 56792 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_1.csv]...
Sampled 56374 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Beginning_January_2018)_2022_2.csv]...
Sampled 51995 data
Processing [On_Time_Marketing_Carrier_On_Time_Performance_(Be

In [5]:
combin_df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
232411,2021,4,10,22,5,2021-10-22,G4,G4,20368,G4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
586260,2021,4,10,16,6,2021-10-16,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2173,2021,4,10,3,7,2021-10-03,DL,DL_CODESHARE,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
217974,2021,4,10,11,1,2021-10-11,F9,F9,20436,F9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
439963,2021,4,10,7,4,2021-10-07,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [6]:
# 看看时间是否对的上
combin_df.groupby(['Year','Month']).count()

Quarter  DayofMonth  DayOfWeek  FlightDate  \
Year Month                                               
2021 10       59537       59537      59537       59537   
     11       57669       57669      57669       57669   
     12       58024       58024      58024       58024   
     7        61570       61570      61570       61570   
     8        61149       61149      61149       61149   
     9        56792       56792      56792       56792   
2022 1        56374       56374      56374       56374   
     2        51995       51995      51995       51995   
     3        59054       59054      59054       59054   
     4        58029       58029      58029       58029   
     5        60295       60295      60295       60295   
     6        60206       60206      60206       60206   
     7        61879       61879      61879       61879   

            Marketing_Airline_Network  \
Year Month                              
2021 10                         59537   
     11                         57669   
     12                         58024   
     7                          61570   
     8                          61149   
     9                          56792   
2022 1                          56374   
     2                          51995   
     3                          59054   
     4                          58029   
     5                          60295   
     6                          60206   
     7                          61879   

            Operated_or_Branded_Code_Share_Partners  DOT_ID_Marketing_Airline  \
Year Month                                                                      
2021 10                                       59537                     59537   
     11                                       57669                     57669   
     12                                       58024                     58024   
     7                                        61570                     61570   
     8                                        61149                     61149   
     9                                        56792                     56792   
2022 1                                        56374                     56374   
     2                                        51995                     51995   
     3                                        59054                     59054   
     4                                        58029                     58029   
     5                                        60295                     60295   
     6                                        60206                     60206   
     7                                        61879                     61879   

            IATA_Code_Marketing_Airline  Flight_Number_Marketing_Airline  \
Year Month                                                                 
2021 10                           59537                            59537   
     11                           57669                            57669   
     12                           58024                            58024   
     7                            61570                            61570   
     8                            61149                            61149   
     9                            56792                            56792   
2022 1                            56374                            56374   
     2                            51995                            51995   
     3                            59054                            59054   
     4                            58029                            58029   
     5                            60295                            60295   
     6                            60206                            60206   
     7                            61879                            61879   

            Originally_Scheduled_Code_Share_Airline  ...  Div5Airport  \
Year Month                                           ...                
2021 10               

In [7]:
combin_df.to_csv('COMBINED_flight_data_202107_to_202207.csv', index = False)

## 飞机型号数据关联


来源：[ICAO_DB](https://github.com/RobAltenburg/icao_db)

该Github项目收集[国际民用航空组织](https://www.icao.int/sustainability/Pages/Databases.aspx)数据，存储为sqlite数据库

数据库中含有353043条记录，直接关联速度较慢，这里先把flight数据集中出现的tail_num提取出来，再去数据库查找对应记录，保存为csv文件。最后用csv和flight关联。

In [2]:
#读取 2021-07 ~ 2022-07的整合数据
df = pd.read_csv('COMBINED_flight_data_202107_to_202207.csv',low_memory=False, dtype='unicode')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762573 entries, 0 to 762572
Columns: 120 entries, Year to Unnamed: 119
dtypes: object(120)
memory usage: 698.2+ MB


In [4]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2021,4,10,22,5,2021-10-22,G4,G4,20368,G4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2021,4,10,16,6,2021-10-16,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,2021,4,10,3,7,2021-10-03,DL,DL_CODESHARE,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,2021,4,10,11,1,2021-10-11,F9,F9,20436,F9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,2021,4,10,7,4,2021-10-07,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [11]:
#连接数据库
import sqlite3
con = sqlite3.connect("ICAO/icao.db")
cur = con.cursor()


#新建飞机型号csv
with open('aircraft_model.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(["model_s", "tail_num", "mfg", "model","updated","comment"])

    
#取所有唯一tail_num
uni_tail_numbers = df['Tail_Number'].unique()

for tail_num in uni_tail_numbers:
    res = cur.execute("SELECT * FROM aircraft where tail_num = '{}'".format(tail_num))
    res = res.fetchone()
    
    #若取到数据
    if res:
        #添加一行  
        with open('aircraft_model.csv', 'a+') as file:
            writer = csv.writer(file)
            writer.writerow(res)

con.close()

In [12]:
aircraft = pd.read_csv('aircraft_model.csv')

print("Unique  tail_num: {}\nMatched tail_num: {}\nMatched rate: {:.4f}".format(len(uni_tail_numbers),len(aircraft),len(aircraft)/len(uni_tail_numbers)))

Unique  tail_num: 5943
Matched tail_num: 4263
Matched rate: 0.7173


**NOTICE**:飞机型号数据存在一定缺失值

In [13]:
flight_with_model = df.merge(aircraft, how='left', left_on=['Tail_Number'], right_on=['tail_num'])
flight_with_model.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,...,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119,model_s,tail_num,mfg,model,updated,comment
0,2021,4,10,22,5,2021-10-22,G4,G4,20368,G4,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,4,10,16,6,2021-10-16,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,N,NaN,a575e8,N451AW,BOMBARDIER INC,CL-600-2B19,1.464739e+09,NaN
2,2021,4,10,3,7,2021-10-03,DL,DL_CODESHARE,19790,DL,...,NaN,NaN,N,NaN,a399b9,N331CA,BOMBARDIER INC,CL-600-2C10,1.464739e+09,NaN
3,2021,4,10,11,1,2021-10-11,F9,F9,20436,F9,...,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021,4,10,7,4,2021-10-07,UA,UA_CODESHARE,19977,UA,...,NaN,NaN,N,NaN,abbb77,N855RW,NaN,E170,1.464739e+09,Emb.170-100SE


In [14]:
# 保存带有飞机型号的航班数据
flight_with_model.to_csv('COMBINED_MODEL_flight_data_202107_to_202207.csv', index = False)

## 天气数据关联
来源：[Meteostat](https://meteostat.net/en/)

该平台收集政府天气数据，并提供[API接口](https://rapidapi.com/meteostat/api/meteostat/)访问  *(3 requests per second for free use)*

其小时为单位天气数据字段如下：

The response body includes the following properties. Please note that all units mentioned below refer to the default units setting.

|col|desc|
|------|------|
|Parameter|Description	Type|
|time|Time (YYYY-MM-DD hh:mm:ss) of observation	String|
|temp|The air temperature in °C	Float|
|dwpt|The dew point in °C	Float|
|rhum|The relative humidity in percent (%)	Integer|
|prcp|The one hour precipitation total in mm	Float|
|snow|The snow depth in mm	Integer|
|wdir|The wind direction in degrees (°)	Integer|
|wspd|The average wind speed in km/h	Float|
|wpgt|The peak wind gust in km/h	Float|
|pres|The sea-level air pressure in hPa	Float|
|tsun|The one hour sunshine total in minutes (m)	Integer|
|coco|The weather condition code	Integer|

https://dev.meteostat.net/api/point/hourly.html#response

**天气数据关联说明**

1. 首先根据DepartCity获取起飞城市经纬度
2. 根据经纬度调用Metostat接口，获取小时数据
3. 将航班数据的**小时**提出来，使用【日期，小时，城市】与天气表进行关联

**问题**

1. 天气数据信息有限，可能只有风速、气压会比较有用？降雨量、降水量缺失值太多没法用
2. 城市太多了，part one有300多个城市，接口一次只能获取一个城市一个月的数据，获取全部天气数据不知道接口会不会被封禁
3. 没有考虑将落地以及航班沿线地区的天气情况，可能影响准确性

In [7]:
#读取 带有飞机型号的整合数据
df = pd.read_csv('COMBINED_MODEL_flight_data_202107_to_202207.csv',low_memory=False, dtype='unicode')

In [13]:
def get_location(loc_name):
    """
    根据城市名获取城市经纬度
    """
    geolocator = Nominatim(user_agent="geoapiExercises")
    address=geolocator.geocode(loc_name)
    
    if address:
        return address.latitude ,address.longitude
    else:
        logging.debug("Couldn't find {}...".format(loc_name))
        return 0



def get_hourly_weather(lat, lon, start, end, tz="America/Toronto"):
    """
    lat, lon: 经纬度
    start, end: 时间范围， YYYY-MM-DD格式
    tz: 时区
    """
    url = "https://meteostat.p.rapidapi.com/point/hourly?"
    payload = {
        "lat":     lat,
        "lon":     lon,
        "start":   start,
        "end":     end,
        "tz":      tz
    }

    #请求密钥，悠着点请求再封没号了
    headers = {
      'x-rapidapi-host': 'meteostat.p.rapidapi.com',
        #密钥找我要
      'x-rapidapi-key' : 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
    }

    try:
        response = requests.request("GET", url, headers=headers, params=payload)
        return json.loads(response.text)
    
    except e as Exception:
        return "天气数据请求失败！\n" + str(e)

In [10]:
#取独立城市名
unique_city_name = df['OriginCityName'].unique()
unique_city_name[:10]

array(['Grand Rapids, MI', 'White Plains, NY', 'Indianapolis, IN',
       'Denver, CO', 'Sarasota/Bradenton, FL', 'Fort Lauderdale, FL',
       'Phoenix, AZ', 'Washington, DC', 'Dallas/Fort Worth, TX',
       'Chicago, IL'], dtype=object)

In [11]:
print(' Unique City numbers: {}\n Query months numbers: {}\n Total request times: {}'.format(len(unique_city_name), 13,len(unique_city_name)*13 ))

 Unique City numbers: 372
 Query months numbers: 13
 Total request times: 4836


需要4836次请求才能获取全部天气数据

In [8]:
#加载字典
with open('city_location.json','r',encoding='utf-8') as f:
    unique_city_location = json.load(f)

#取独立城市名
unique_city_name = df['OriginCityName'].unique()
unique_city_name[:10]

array(['Grand Rapids, MI', 'White Plains, NY', 'Indianapolis, IN',
       'Denver, CO', 'Sarasota/Bradenton, FL', 'Fort Lauderdale, FL',
       'Phoenix, AZ', 'Washington, DC', 'Dallas/Fort Worth, TX',
       'Chicago, IL'], dtype=object)

In [ ]:
# 先遍历所有城市，把城市地理信息存进字典
# 已全部遍历，该CELL不用执行

for city_name in unique_city_name:
    if city_name not in unique_city_location.keys():
        try:
            #print(city_name)
            unique_city_location[city_name] = get_location(city_name)
            logging.info('Process CITY {}'.format(city_name))
            time.sleep(random.randint(3,10))
            
        except:
            logging.info('ERROR CITY {}'.format(city_name))

In [ ]:
"""
以下地名存在斜杠无法识别，单独处理
    Mission/McAllen/Edinburg, TX
    Cedar Rapids/Iowa City, IA
    Harlingen/San Benito, TX
    Bristol/Johnson City/Kingsport, TN
    Iron Mountain/Kingsfd, MI
    Sun Valley/Hailey/Ketchum, ID
    Riverton/Lander, WY
    Newburgh/Poughkeepsie, NY
    Saginaw/Bay City/Midland, MI
    New Bern/Morehead/Beaufort, NC
"""

for k,v in unique_city_location.items():
    if v ==0:
        try:
            #存在斜杠的，取第一个城市名，验证经纬度基本一致
            first_city_name = k.split('/')[0]

            unique_city_location[k] = get_location(first_city_name)
        except:
            logging.info('ERROR CITY {}'.format(city_name))

In [8]:
# 检查是否为372
len(unique_city_location)

372

In [9]:
#保存字典到本地
with open('city_location.json','w',encoding='utf-8') as f:
    json.dump(unique_city_location,f)

### 天气查询逻辑
从2021-07到2022-7，一个一个月份，一个一个城市进行查询

比如只有A,B,C三个城市：

    request-> 2021-07 - city A
    request-> 2021-07 - city B
    request-> 2021-07 - city C
    request-> 2021-08 - city A
    request-> 2021-08 - city B
    request-> 2021-08 - city C
    ……
    request-> 2022-07 - city A
    request-> 2022-07 - city B
    request-> 2022-07 - city C

In [5]:
# 导入日期处理相关库
import calendar
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

In [14]:
# 已请求完整2021-7 ~ 2022-7 ，无需执行

weather = pd.DataFrame()
count = 0
date_init = date(2021, 7, 1)
#标准化列明，防止concat出错
std_columns = ['time', 'temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt',
                           'pres', 'tsun', 'coco', 'locName', 'lat', 'long', 'stationCode']

for m in range(13):
    date_process = date_init + relativedelta(months=+m)
    #print(date_process)  
    
    #每月月初日期
    mon_start_date = date_process.strftime("%Y-%m-%d")
    
    #每月月末日期
    end_day = calendar.monthrange(date_process.year, date_process.month)[1] 
    mon_end_date = date_process.strftime("%Y-%m") + "-" + str(end_day)
    
    print("processing Date: {}...".format(mon_start_date))
    logging.debug("processing Date: {}...".format(mon_start_date))
    #print(mon_start_date, mon_end_date)
    

    for loc_name in unique_city_name:
        
        logging.debug("Date: {} City: {}...".format(mon_start_date,loc_name))
        #print("Date: {} City: {}...".format(mon_start_date,loc_name))
        
        #从字典获取城市geo
        lat = unique_city_location[loc_name][0]
        long =  unique_city_location[loc_name][1]
        
        
        try:
            data = get_hourly_weather(lat, long, mon_start_date, mon_end_date)

            stationCode = data['meta']['stations']
            
            #json转df
            tmp = pd.json_normalize(data['data'])
            tmp['locName'] = loc_name
            tmp['lat'] = lat
            tmp['long'] = long
            
            # "/"区分多station情况
            tmp['stationCode'] = "/".join(stationCode)

            #睡0.5秒防止频率太快封IP  (3 requests per second for free use)
            time.sleep( 0.1 * random.randint(4,6))
            
            #标准化列名
            tmp.columns = std_columns

            weather = pd.concat([weather,tmp], axis=1)

            count += 1
            if count%10==0:
                print("[{}]Date: {} City: {}...".format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) ,mon_start_date,loc_name))
        
        except Exception as e :
            print('ERROR at date {}, city {} \n {}'.format(mon_start_date,loc_name,e))
            logging.info('[{}]ERROR at date {}, city {} \n {}'.format(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()) ,mon_start_date,loc_name,e))
            weather.to_csv('WEATHER_ALL_error.csv', index = False)
            
weather.to_csv('WEATHER_ALL_STD.csv', index = False)

processing Date: 2021-07-01...


In [18]:
weather

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco,locName,lat,long,stationCode
0,2021-07-01 00:00:00,21.1,17.9,82.0,0.0,None,290.0,13.0,None,1015.9,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
1,2021-07-01 01:00:00,21.1,17.9,82.0,0.0,None,290.0,7.6,None,1015.4,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
2,2021-07-01 02:00:00,20.6,17.8,84.0,0.0,None,300.0,11.2,None,1015.6,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
3,2021-07-01 03:00:00,20.0,17.8,87.0,0.0,None,320.0,9.4,None,1015.2,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
4,2021-07-01 04:00:00,19.4,17.2,87.0,0.0,None,330.0,9.4,None,1015.0,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2021-07-31 19:00:00,23.9,14.0,54.0,0.0,None,260.0,18.4,None,1013.7,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
740,2021-07-31 20:00:00,22.8,14.4,59.0,0.0,None,250.0,16.6,None,1013.6,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
741,2021-07-31 21:00:00,21.7,15.1,66.0,0.0,None,250.0,14.8,None,1013.2,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636
742,2021-07-31 22:00:00,21.1,15.6,71.0,0.0,None,230.0,13.0,None,1013.0,None,NaN,"Grand Rapids, MI",42.96324,-85.667864,72635/KBIV0/KY700/72636


## 读取天气数据，开始关联

In [3]:
weather = pd.read_csv('WEATHER_ALL_STD.csv',low_memory=False, dtype='unicode')

In [67]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3588312 entries, 0 to 3588311
Data columns (total 16 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   time         object
 1   temp         object
 2   dwpt         object
 3   rhum         object
 4   prcp         object
 5   snow         object
 6   wdir         object
 7   wspd         object
 8   wpgt         object
 9   pres         object
 10  tsun         object
 11  coco         object
 12  locName      object
 13  lat          object
 14  long         object
 15  stationCode  object
dtypes: object(16)
memory usage: 438.0+ MB


In [68]:
weather.drop(['snow','tsun'], axis=1, inplace = True)
#815980行有问题，去除
weather.drop(815980, inplace = True)
weather.head()

,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,coco,locName,lat,long,stationCode
0,2021-07-01 00:00:00,21.1,17.9,82.0,0.0,290.0,13.0,NaN,1015.9,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636
1,2021-07-01 01:00:00,21.1,17.9,82.0,0.0,290.0,7.6,NaN,1015.4,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636
2,2021-07-01 02:00:00,20.6,17.8,84.0,0.0,300.0,11.2,NaN,1015.6,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636
3,2021-07-01 03:00:00,20.0,17.8,87.0,0.0,320.0,9.4,NaN,1015.2,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636
4,2021-07-01 04:00:00,19.4,17.2,87.0,0.0,330.0,9.4,NaN,1015.0,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636


In [69]:
weather.dropna(subset=['time'], inplace = True)

In [70]:
#提取日期、时间
weather['Date'] = weather['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d'))
weather['Hour'] = weather['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S').strftime('%H'))
#去重
weather.drop_duplicates(subset=['time', 'stationCode'], keep='last', inplace=True)
weather.head()

,time,temp,dwpt,rhum,prcp,wdir,wspd,wpgt,pres,coco,locName,lat,long,stationCode,Date,Hour
0,2021-07-01 00:00:00,21.1,17.9,82.0,0.0,290.0,13.0,NaN,1015.9,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636,2021-07-01,00
1,2021-07-01 01:00:00,21.1,17.9,82.0,0.0,290.0,7.6,NaN,1015.4,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636,2021-07-01,01
2,2021-07-01 02:00:00,20.6,17.8,84.0,0.0,300.0,11.2,NaN,1015.6,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636,2021-07-01,02
3,2021-07-01 03:00:00,20.0,17.8,87.0,0.0,320.0,9.4,NaN,1015.2,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636,2021-07-01,03
4,2021-07-01 04:00:00,19.4,17.2,87.0,0.0,330.0,9.4,NaN,1015.0,NaN,"Grand Rapids, MI",42.9632405,-85.6678639,72635/KBIV0/KY700/72636,2021-07-01,04


In [71]:
#时间补足为4位
df['CRSDepTime'] = df['CRSDepTime'].apply(lambda x: str(x).zfill(4))

#取小时
df['DepHour'] = df['CRSDepTime'].apply(lambda x: str(x)[:2])
df[['OriginCityName','FlightDate','CRSDepTime','DepTime','DepHour']]

,OriginCityName,FlightDate,CRSDepTime,DepTime,DepHour
0,"Grand Rapids, MI",2021-10-22,0827,0824,08
1,"White Plains, NY",2021-10-16,0630,0621,06
2,"Indianapolis, IN",2021-10-03,1410,1406,14
3,"Denver, CO",2021-10-11,2245,2241,22
4,"Sarasota/Bradenton, FL",2021-10-07,1440,1433,14
...,...,...,...,...,...
762568,"Indianapolis, IN",2022-07-29,1930,1923,19
762569,"San Francisco, CA",2022-07-01,0625,0621,06
762570,"Tampa, FL",2022-07-11,2115,2108,21
762571,"Washington, DC",2022-07-03,1455,1522,14


In [72]:
#关联一下试试
# 以航班表为底表左关联天气表，使用【日期，小时，城市】进行关联，需注意天气表小时粒度要保证无重复值
res = df.merge(weather, how='left', left_on=['FlightDate', 'DepHour','OriginCityName'], right_on=['Date', 'Hour','locName'])

In [74]:
res.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762573 entries, 0 to 762572
Columns: 143 entries, Year to Hour
dtypes: object(143)
memory usage: 837.8+ MB


In [75]:
res.to_csv('Flight_data_with_MODEL_&_WEATHER.csv', index = False)